# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import numpy as np
import pandas as pd
import pickle
import sklearn
import sqlite3
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as fscore
from sklearn.base import BaseEstimator, TransformerMixin
from collections import Counter

import warnings  # requested to managed get_scores with this version of sklearn
                 # according to https://stackoverflow.com/questions/43162506/undefinedmetricwarning-f-score-is-ill-defined-and-being-set-to-0-0-in-labels-wi
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
def load_data():
    '''
    DESCRIPTION
        Loading data from database and split it into messages (X) and categories (Y)
    INPUT
        nil
    OUTPUT
        X is a numpy array of all messages only (strings)
        Y is a numpy array of all other categories relates to these messages (except 'original' messages -Fr, UK, US ...-
          in column #2, and except 'genre' in column #3
        category_names is names for Y categories
    '''
    
    engine = create_engine('sqlite:///InsertDatabaseName.db')
    df = pd.read_sql_table(table_name='InsertTableName', con=engine)    
    X = df["message"].values  # get column of messages
    Y = np.asarray(df[df.columns[4:]].values) # get categories columns
    Y = Y.astype(int)  # convert all Y values to int  # help from https://knowledge.udacity.com/questions/59448
    category_names = df.columns[4:]
    return X, Y, category_names

X, Y, category_names = load_data()

In [3]:
print('X', len(X), type(X), '\n', X)

X 26216 <class 'numpy.ndarray'> 
 ['Weather update - a cold front from Cuba that could pass over Haiti'
 'Is the Hurricane over or is it not over'
 'Looking for someone but no name' ...,
 "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families."
 'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.'
 'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.']


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    '''
    DESCRIPTION
        text tokenization
    INPUT
        text is a list of messages (strings)
    OUTPUT
        clean_tokens is XXXX as result of the tokenization, which includes
         - normalizing the text, i.e. keep only spaces, letters and figures
         - Tokenization itself
         - Stop words (removal of all not english words; i.e. the X columns is already a conversion of
           messages from various languages (Fr, En ...) to english, thus no need to consider another language
         - Lemmatization
           including a change for Lower casing + remove leading and trailing spaces

    '''
    
    # Normalize, remove all that is not letters and figures
    pure_text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    
    # split sentences into word
    tokens = word_tokenize(pure_text)
    
    # remove stop words from the sentences
    stop_words = stopwords.words('english')  # assume that there is only english sentences
    stop_word = [word for word in tokens if word not in stop_words]
    
    # normalize words keeping the source word according to the context
    #  and get only lower characters and remove leading and trailing characters
    clean_tokens = []
    lemmatizer = WordNetLemmatizer()
    for word in stop_word:
        lemmatized = lemmatizer.lemmatize(word).lower().strip()
        clean_tokens.append(lemmatized)    
    
    # attached the type of word to the word
    #tagged = nltk.pos_tag(lemmatized)  # not used for the time being                    
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def main(X, y):
    '''
    DESCRIPTION
        classify and predict messages through pipelines with RandomForestClassifier
    INPUT
        X is a list of X_train and X_test
        y is a list of y_train and y_test
    OUTPUT
        y_pred is the predictions for X_test based on the pipeline operations: tokenization, Fit/tranform and classifier
        pipeline is the Pipeline defined for classification of data

    comment:
    Initially, load and split of data into train and test sets was implemented directly into the main
     but it was finally exited to comply with the request below to handle actions separately
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    print('Train_size:', len(X_train), '|', 100*len(X_train) / len(X), "%", '| type:', type(X_train))
    print('Test_size : ', len(X_test), '|', 100*len(X_test) / len(X), "%", '| type:', type(X_train))
    '''
    
    X_train, X_test = X
    y_train, y_test = y
    
    # Build the pipeline
    # - using MultiOutputClassifer link and refering to https://knowledge.udacity.com/questions/158218
    print('PIPELINE')
    pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),        
            #  ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 10, n_jobs = -1)))
            ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])
    # MultiOutputClassifier (fit one classifier per target):
    #  n_estimators = 10
    #  n = -1  means using all available processes / threads

    # train classifier
    print('CLASSIFIER')
    pipeline.fit(X_train, y_train)  # Final estimator of classifier type

    # predict on test data
    print('PREDICT')
    y_pred = pipeline.predict(X_test)

    return y_pred, pipeline

#main()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Get data
X, y, category_names = load_data()

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)
print('Train_size:', len(X_train), '|', 100*len(X_train) / len(X), "%", '| type:', type(X_train))
print('Test_size : ', len(X_test), '|', 100*len(X_test) / len(X), "%", '| type:', type(X_train))

# Train pipeline
y_pred, pipeline = main([X_train, X_test], [y_train, y_test])

Train_size: 19662 | 75.0 % | type: <class 'numpy.ndarray'>
Test_size :  6554 | 25.0 % | type: <class 'numpy.ndarray'>
PIPELINE
CLASSIFIER
PREDICT


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
print('sklearn version', sklearn.__version__)  # = 0.19.1
print('\n')
    
def get_scores(y_test, y_pred):
    '''
    DESCRIPTION
        classify the result between y_test and y_pred
    INPUT
        y_test is an array of y test data
        y_pred is an array of predictions of y test data
    OUTPUT
        reports is the textual report (string) of the classification report result
    '''

    # For f1_score;
    # Initially refering to refering to https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
    # F1_score = f1_score(y_test, y_pred, average='macro')  # , labels=np.unique(y_pred)
    #  but it raises "ValueError: multiclass-multioutput is not supported"
    # so refering to https://stackoverflow.com/questions/33326704/scikit-learn-calculate-f1-in-multilabel-classification

    mlb = MultiLabelBinarizer().fit(y_test)
    f1_score_mlb = f1_score(mlb.transform(y_test),
             mlb.transform(y_pred),
             average='macro')
    print('f1_score_mlb:', f1_score_mlb)
    print('\n')

    # Then using the classificatio_report mentioned for the exercice:
    # refering to https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
    reports = []
    for i in range(len(category_names)):
        report = classification_report(np.array(y_test)[i], np.array(y_pred)[i])
        # zero_division=0 & output_dict=True : available only with New in version 0.20.
        # print('report', i, '\t', report)
        reports.append(report)
        # report_score = fscore(y_test, y_pred, average='weighted')
        # print('report_score', i, '\t', report_score)  cancelled because "ValueError: multiclass-multioutput is not supported"
    return reports


def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

    
reports_class_report = get_scores(y_test, y_pred)

# display_results(y_test, y_pred)  # not used because of a "ValueError: multiclass-multioutput is not supported"

sklearn version 0.19.1


F1_score: 0.746838067907




In [8]:
def get_scores_dict(report_txt, titles):
    '''
    DESCRIPTION
        Transform a text of report into a global dictionary
    INPUT
        report_txt is report under text format
        titles is the list of names for every results table
    OUTPUT
        dico is a dictionary of titles with related result per values 0, 1, total and related scores
        {title_1: {'0': {'precision': x, 'recall: x', 'f1-score': x, 'support': x}, '1': {...}, 'total': {...} }, title_2: ...}
    
    A line of the report contains:
       report A      precision  recall  f1-score  support
               0        x          x        x        x
               1        x          x        x        x
               total    x          x        x        x
    Sometimes, 0 or 1 my be not present
    '''
        
    lst = []  # initiate a list of list i.e. a list (lst) containing all words of every table contained in a related list (mots)
    
    # Read line by line of the report; one table result is contained on a line
    for tab in report_txt:
        # create a table with all elements, usung space as separator
        li = tab.split(' ')
        mots = []  # initiate a list of word and figure that will be extract from the line
        # Read the list of word read on the line
        for i, elt in enumerate(li):
            if elt != '':  # when word contains characters
                if i == 0:            # when this is the first word
                    mots.append(elt)  # - store this word in the list mots
                elif '\n' in elt:     # when the word contains a line break, indicating the end a of sub-line,
                                      #  it's time to change of sub-list
                    elt_f = elt.split('\n')[0]  # - get the first part of the word 
                    mots.append(elt_f)          # - and store it in the last list
                    lst.append(mots)            # - and store this list mtos into the list lst, before initiating a new list mots
                    elt_s = elt.split('\n')[1].replace('\n','')  # - get the second part of the word
                    if elt_s != '':             # - when the second of the word contains characters
                        mots = [elt_s]          # - store this new word into the new list mots
                    else:                       # - otherwise
                        mots = []               # - initiate a empty list mots
                else:
                    mots.append(elt)   # otherwise (not the first word, not the last word of a sub-line), then store the word
                                       #  into the list mots
        lst.append(mots)  # Finally, store every mots (containing words of every sub-line)
    
    # Now that we have list and sub-list of words of the report
    #  we gonna get values per label
    dico = dict()
    start_cycle = -1
    di = dict()
    k = -1
    # Read every sub-list of words of the list
    for i, t in enumerate(lst):
        label = ''        
        values = t[-4:]  # Get last four values of every line
        if len(values) > 0:  # if there are values
            if values[0] == 'precision': # detect the word 'precision' as starter
                k += 1  # iterate k as index to join title and related data
                start_cycle = i  # store i
                if len(di) > 0:          # if we fulfill di (dict) during previous lines
                    dico[titles[k]] = di #  then we store it in dico 
                di = dict()              # any way, at thi stage of the starter, we initiate dict di
                                         # di is a dict to store every result values 
            else:  # when we are in another line that the starter
                if i > start_cycle: 
                    label = label.join(t[:(len(t)-4)]).replace('/', '') # we get the label of this line
                    # and then we get score values for every category of result, storing that in a dictionary d
                    d = dict()  # d for storing scores of a label
                    d['precision'] = values[0]
                    d['recall'] = values[1]
                    d['f1-score'] = values[2]
                    d['support'] = values[3]
                    di[label] = d  # when we get all scores, we store it into dictionary di, di = scores for all labels of a table

    return dico


def get_feature_values(dico, feature):
    '''
    DESCRIPTION
        Reading a dictionary of result (from get_scores), we focus on a feature of results to get these values
    INPUT
        dico is a dictionary of results, multi tables, multi-lines with result features for every line
    OUTPUT
        dico_feat is a dictionary of the result-score for the selected result-feature
        {title_1: [0_score, 1_score, total_score], ...}, nan is no value
    '''
        
    dico_feat = dict()
    # Read every table of result
    for i, j in dico.items():
        k0, k1, k2 = np.nan, np.nan, np.nan  # initiate scores for the selected result-feature
        
        # Read every label and relates scores of a table
        for k, m in j.items():
            #  get score for every label
            if k == '0':
                try:
                    k0 = float(m[feature])
                except:
                    k0 = np.nan
            elif k == '1':
                try:
                    k1 = float(m[feature])
                except:
                    k1 = np.nan
            else:
                try:
                    k2 = float(m[feature])
                except:
                    k2 = np.nan
        
        # store scores into a dictionary
        dico_feat[i] = [k0, k1, k2]
    return dico_feat


def get_list_value(lst, pos):
    try:
        r = lst[pos]
    except:
        r = np.nan
    return r


def dict_to_df(dico):
    '''
    DESCRIPTION
        transform a dictionary of scores (from get_feature_values) into a dataframe
    INPUT
        dico is a dictionary of the score per title
              under format {title_1: [0_score, 1_score, total_score], ...}, nan is no value
    OUTPUT
        df is a dataframe with these scores, under format
        columns = items, '0', '1', 'total' ; rows = titles ; values = score-values
    '''
    
    col = ['items', '0', '1', 'total']
    items, zero, un, total = [], [], [], []
    for i, j in dico.items():
        items.append(i)
        # It's possible not to have a value for 0, 1 or total, so we have to manage it by proposing an alternate solution
        zero.append(get_list_value(j, 0))
        un.append(get_list_value(j, 1))
        total.append(get_list_value(j, 2))

    d = {'items': items, '0': zero, '1': un, 'total':total}
    df = pd.DataFrame(data=d)
    df[['0', '1', 'total']].astype(float)  # change type values to float
    return df


def get_f1_score(report, titles, result_feature='f1-score'):
    '''
    DESCRIPTION
        Transform a report of scoresinto a dataframe of the selected result-feature (here: f1-score)
    INPUT
        report is the report of scores (string)
        titles is the list of names of all parameters
        result_feature is the name of the result-feature that we want to get ; f1-score per default
    OUTPUT
        df_feature is a dataframe with result-scores of the selected result-feature for all labels and parameters
    '''
    dico = get_scores_dict(report, titles)  # Transform a text of report into a global dictionary
    dico_feature = get_feature_values(dico, result_feature)  # Reading a dictionary of result (from get_scores),
                                                             #  we focus on a feature of results to get these values
    df_feature = dict_to_df(dico_feature)  # transform a dictionary of scores (from get_feature_values) into a dataframe
    
    return df_feature


In [9]:
# display the f1-score for classification Report model
df_scores_class_report = get_f1_score(reports_class_report, category_names)
print(df_scores_class_report.head())
f1_score_cr = df_scores_class_report["total"].mean()
print('f1_score_cr:', f1_score_cr) 

,items,0,1,total
0,request,0.98,0.86,0.97
1,offer,0.98,0.86,0.97
2,aid_related,1.00,1.00,1.00
3,medical_help,0.94,0.33,0.87
4,medical_products,0.96,0.40,0.89


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# According to https://knowledge.udacity.com/questions/451426, I reduced the parameters to the minimum only to show
#  the principle of the Grid Search, not to get the best optimization. 
parameters = {
    'clf__estimator__n_estimators': [2]
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs= -1)

cv.fit(X_train, y_train)
y_pred_cv = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
mlb = MultiLabelBinarizer().fit(y_test)
F1_score_cv = f1_score(mlb.transform(y_test),
         mlb.transform(y_pred_cv),
         average='macro')

# Get the f1-score from the MultiLabelBinarizer model
reports_GridSearch = get_scores(y_test, y_pred_cv)

F1_score: 0.677046692626




In [12]:
# display the f1-score for MultiLabelBinarizer model
df_scores_GridSearch = get_f1_score(reports_GridSearch, category_names)
df_scores_GridSearch.head()

,items,0,1,total
0,request,0.98,0.86,0.97
1,offer,0.98,0.86,0.97
2,aid_related,1.00,1.00,1.00
3,medical_help,0.96,0.40,0.89
4,medical_products,0.96,0.40,0.89


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        # print('\nsentence_list:', sentence_list)
        for sentence_raw in sentence_list:
            sentence = re.sub(r'[^a-zA-Z0-9 ]', '', sentence_raw)
            sentence.rstrip()
            frequence = Counter(sentence)
            # print('  frequence:', frequence[' '])
            # print('  sentence len:', len(sentence))
            # print('  sentence: [', sentence, ']')
            if len(sentence) > frequence[' ']:
                pos_tags = nltk.pos_tag(tokenize(sentence))
                # print('  pos_tags:', pos_tags)
                # print('  pos_tags len:', len(pos_tags))
                if len(pos_tags) > 0:
                    first_word, first_tag = pos_tags[0]
                    if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                        return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [14]:
from sklearn.pipeline import FeatureUnion


# help: https://www.kaggle.com/code/suveesh/create-custom-transformers/notebook


def main_2(X, y):
    '''
    DESCRIPTION
        classify and predict messages through pipelines with StartingVerbExtractor
    INPUT
        X is a list of X_train and X_test
        y is a list of y_train and y_test
    OUTPUT
        y_pred is the predictions for X_test based on the pipeline operations: tokenization, Fit/tranform and classifier
        pipeline is the Pipeline defined for classification of data
    '''
    
    X_train, X_test = X
    y_train, y_test = y
    
    # Build the pipeline
    # - using MultiOutputClassifer link and refering to https://knowledge.udacity.com/questions/158218
    print('PIPELINE')
    '''
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('nlp_pipeline', Pipeline([
                ('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor()),
         ])),
         ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    '''
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
        
    # MultiOutputClassifier (fit one classifier per target):
    #  n_estimators = 10
    #  n = -1  means using all available processes / threads

    # train classifier
    print('CLASSIFIER')
    pipeline.fit(X_train, y_train)  # Final estimator of classifier type

    # predict on test data
    print('PREDICT')
    y_pred = pipeline.predict(X_test)
    # print('y_pred type:', type(y_pred))

    return y_pred, pipeline


In [15]:
# Train pipeline
y_pred_sve, pipeline_2 = main_2([X_train, X_test], [y_train, y_test])

print()

# Get the f1-score from the StartingVerbExtractor model
reports_SVE = get_scores(y_test, y_pred_sve)

# display the f1-score for StartingVerbExtractor model
df_scores_SVE = get_f1_score(reports_SVE, category_names)
df_scores_SVE.head()

PIPELINE
CLASSIFIER
PREDICT

F1_score: 0.759351309047




,items,0,1,total
0,request,0.98,0.86,0.97
1,offer,0.97,0.67,0.94
2,aid_related,1.00,1.00,1.00
3,medical_help,0.94,0.33,0.87
4,medical_products,0.97,0.67,0.94


### 9. Export your model as a pickle file

In [16]:
pickle.dump(main, open('model_main.pkl', 'wb'))
pickle.dump(cv, open('model_cv.pkl', 'wb'))
pickle.dump(main_2, open('model_2.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [20]:
# No template file "train.py"
# Udacity Help:
# https://knowledge.udacity.com/questions/167584
# https://knowledge.udacity.com/questions/251588

# train_classifier is on the Worskpace IDE